In [1]:
import pandas as pd
import numpy as np
import requests 
from io import StringIO

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

plt.style.use('ggplot')

/Users/marc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
cex = pd.read_csv('data/walletbookCEX.csv')
cex = cex[cex['is_cex']==1]
cex.to_csv('Algorand_CEX.csv', index=False)

In [16]:
gov = pd.read_csv('Governance/gp13.csv')
gov['TotalCommitedALGO']=gov['TotalCommitedALGO']/1000000

gov[(gov['TotalCommitedALGO'] > 100000)]['TotalCommitedALGO'].sum()

1563325917.707233

In [5]:
cex_list = ['Bitmax', 'Binance', 'BingX', 'Bitbox', 
 'Bitfinex', 'BitGo', 'Bitkub', 'BitPanda',
 'Bitrue', 'Bitso', 'Bitstamp', 'Bittrex',
 'Bitvavo', 'BitZ', 'BKEX', 'Blockchain.com', 
 'BTC Markets', 'BTCTurk', 'ByBit', 'Changelly',
 'Coinbase', 'Coinex', 'Coinlist', 'CoinSpot',
 'Crypto.com', 'Gate.io', 'Hotbit', 'Indodax',
 'Koibanx', 'Korbit', 'Kraken', 'Kucoin',
 'MEXC', 'Netcoins', 'OkCoin', 'OKX',
 'Paribu', 'Phemex', 'ProBit', 'Ripio', 
 'UpHold', 'Upbit', 'Vindax', 'WhiteBIT',
 'AEX', 'Bilaxy', 'Nobitex', 'Xeggex',
 'XT', 'Blofin']



In [22]:

# Define the API endpoint
url = "https://algoanalytics.api.nodely.io/v1/af/entities/history/cex"

username = 'username'
password = 'pass'

# Send a GET request to the API with basic authentication
response = requests.get(url, auth=(username, password), headers={'accept': '*/*'})

if response.status_code == 200:
    # Get the raw text response
    raw_content = response.text
    
    df = pd.read_csv(StringIO(raw_content), delimiter=",")
        
df

,Date,Bitmax,OKX,Bilaxy,Coinex,Hotbit,Bitfinex,Bitbox,Gate.io,BitZ,...,BTCTurk,Bitso,Nobitex,BitGo,BingX,Koibanx,Ripio,Xeggex,Blofin,Total
0,2019-06-18,1.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,1.000000e+00
1,2019-06-19,3.839060e-01,1.890000e-01,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,5.729060e-01
2,2019-06-20,1.580842e+06,1.890000e-01,3224.700335,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,1.584067e+06
3,2019-06-21,1.771463e+06,1.012749e+06,24309.318397,7.528214e+03,1378.446735,5.096099e+05,5453.086642,1.008990e+02,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,3.332592e+06
4,2019-06-22,1.517942e+06,1.580193e+06,15387.963627,1.347107e+04,843.818147,5.294894e+05,5031.814045,8.765459e+04,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,3.750014e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,2024-10-13,1.753007e+05,4.704884e+07,6983.607744,3.469992e+06,575173.467205,6.733342e+06,4645.409785,1.286367e+07,0.1,...,1.843369e+07,5.863680e+06,1.988779e+06,0.198,48518.845570,3.126,5635.828230,45225.619606,119218.300331,2.870505e+09
1945,2024-10-14,1.741756e+05,4.736022e+07,6983.607744,3.476403e+06,559039.552406,6.864389e+06,4645.409785,1.282823e+07,0.1,...,1.843369e+07,5.902627e+06,1.941645e+06,0.198,36971.920794,3.126,5456.054596,50872.090067,108701.944850,2.876796e+09
1946,2024-10-15,1.718180e+05,4.698170e+07,6983.607744,3.553003e+06,592408.189947,6.793350e+06,4645.409785,1.305297e+07,0.1,...,1.843369e+07,5.916218e+06,1.911726e+06,0.198,36971.920794,3.126,7837.469738,45784.983126,133963.742952,2.876524e+09
1947,2024-10-16,1.707086e+05,4.597283e+07,6983.607744,3.559530e+06,618181.472647,6.794452e+06,4645.409785,1.327081e+07,0.1,...,1.843369e+07,5.875810e+06,1.901561e+06,0.198,36971.920794,3.126,7700.988955,48288.942511,128433.106349,2.874574e+09


In [23]:
pivot_df = df
pivot_df['Date'] = pd.to_datetime(pivot_df['Date'])
last_date = pivot_df['Date'].max()

# Step 3: Get the last day balances
last_day_df = pivot_df[pivot_df['Date'] == last_date]

# Step 4: Prepare ranking DataFrame
ranking_data = {'CEX': [], 'Balance': [], '24h%': [], '7d%': [], '30d%': [], '90d%': []}

# Get the list of CEX columns (excluding 'Date')
cex_columns = [col for col in pivot_df.columns if col != 'Date']

# Step 5: Populate the ranking data
for cex in cex_columns:
    # Extract the last day balance
    balance = last_day_df[cex].values[0]
    
    # Add CEX name and Balance
    ranking_data['CEX'].append(cex.replace(' Balance', ''))
    ranking_data['Balance'].append(balance)
    
    # Calculate 24-hour change
    previous_day = last_date - pd.Timedelta(days=1)
    previous_day_balance = pivot_df[pivot_df['Date'] == previous_day][cex]
    
    if not previous_day_balance.empty and previous_day_balance.values[0] != 0:
        change_24h = ((balance - previous_day_balance.values[0]) / previous_day_balance.values[0]) * 100
    else:
        change_24h = None
    ranking_data['24h%'].append(change_24h)
    
    # Calculate percentage changes for 7d, 30d, and 90d
    for days in [7, 30, 90]:
        past_date = last_date - pd.Timedelta(days=days)
        past_balance = pivot_df[pivot_df['Date'] == past_date][cex]
        
        if not past_balance.empty and past_balance.values[0] != 0:
            change = ((balance - past_balance.values[0]) / past_balance.values[0]) * 100
        else:
            change = None
        
        ranking_data[f'{days}d%'].append(change)

# Step 6: Create the ranking DataFrame
ranking_df = pd.DataFrame(ranking_data)

# Step 7: Sort the ranking DataFrame by Balance in descending order
ranking_df.sort_values(by='Balance', ascending=False, inplace=True)
ranking_df.reset_index(drop=True, inplace=True)
ranking_df.to_csv('data/CEX_ranking.csv', index=False)

In [2]:
# Load the data into a DataFrame
df = pd.read_csv('CEX_balances/result.csv')

# Convert 'dt' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Pivot the DataFrame to have 'entity_name' as columns and 'dt' as the index
pivot_df = df.pivot(index='Date', columns='Entity', values='AlgoBalance')

# Create a new column for total balance by summing across entities
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df.reset_index(inplace=True)

In [16]:
pivot_df.to_csv('data/CEX_balances.csv', index=False)

In [15]:
pivot_df['Date'] = pd.to_datetime(pivot_df['Date'])
last_date = pivot_df['Date'].max()

# Step 3: Get the last day balances
last_day_df = pivot_df[pivot_df['Date'] == last_date]

# Step 4: Prepare ranking DataFrame
ranking_data = {'CEX': [], 'Balance': [], '24h%': [], '7d%': [], '30d%': [], '90d%': []}

# Get the list of CEX columns (excluding 'Date')
cex_columns = [col for col in pivot_df.columns if col != 'Date']

# Step 5: Populate the ranking data
for cex in cex_columns:
    # Extract the last day balance
    balance = last_day_df[cex].values[0]
    
    # Add CEX name and Balance
    ranking_data['CEX'].append(cex.replace(' Balance', ''))
    ranking_data['Balance'].append(balance)
    
    # Calculate 24-hour change
    previous_day = last_date - pd.Timedelta(days=1)
    previous_day_balance = pivot_df[pivot_df['Date'] == previous_day][cex]
    
    if not previous_day_balance.empty and previous_day_balance.values[0] != 0:
        change_24h = ((balance - previous_day_balance.values[0]) / previous_day_balance.values[0]) * 100
    else:
        change_24h = None
    ranking_data['24h%'].append(change_24h)
    
    # Calculate percentage changes for 7d, 30d, and 90d
    for days in [7, 30, 90]:
        past_date = last_date - pd.Timedelta(days=days)
        past_balance = pivot_df[pivot_df['Date'] == past_date][cex]
        
        if not past_balance.empty and past_balance.values[0] != 0:
            change = ((balance - past_balance.values[0]) / past_balance.values[0]) * 100
        else:
            change = None
        
        ranking_data[f'{days}d%'].append(change)

# Step 6: Create the ranking DataFrame
ranking_df = pd.DataFrame(ranking_data)

# Step 7: Sort the ranking DataFrame by Balance in descending order
ranking_df.sort_values(by='Balance', ascending=False, inplace=True)
ranking_df.reset_index(drop=True, inplace=True)
ranking_df.to_csv('data/CEX_ranking.csv', index=False)

In [32]:
usdc_holders = pd.read_csv('data/usdc_holders.csv')

walletbook = pd.read_csv('data/walletbook_cex.csv')

usdc_holders2 = usdc_holders.merge(walletbook[['addr', 'label']], left_on='address', right_on='addr', how='left')

# Combine the two columns into one and drop the unnecessary columns
usdc_holders2['label'] = usdc_holders2['label_x'].combine_first(usdc_holders2['label_y'])

# Drop the redundant columns
usdc_holders2 = usdc_holders2.drop(columns=['label_x', 'label_y', 'addr'])

In [34]:
usdc_holders2.to_csv('data/usdc_holders.csv')